In [2]:
import transformers
from datasets import load_dataset
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import torch
import librosa
import os

sr = 16000

## data load

In [11]:
start_time = time.time()
all_data = load_dataset('csv',data_files='./order_speech_ko1000.csv',split='train')
print(time.time()-start_time)

Using custom data configuration default-3a7ac772dc9fef4d
Reusing dataset csv (C:\Users\Mu-jun\.cache\huggingface\datasets\csv\default-3a7ac772dc9fef4d\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


30.802470922470093


## data cleaning

In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [13]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x000001A179641EE8> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1000 [00:00<?, ?ex/s]

## Create Wav2Vec2CTCTokenizer

In [5]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json",
                                 unk_token="[UNK]",
                                 pad_token="[PAD]",
                                 word_delimiter_token="|")

## Create XLSR-Wav2Vec2 Feature Extractor

In [6]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)

In [7]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor,
                              tokenizer=tokenizer)

In [17]:
# from google.colab import drive
# drive.mount('/content/gdrive/')

In [18]:
# processor.save_pretrained("/content/gdrive/MyDrive/wav2vec2-large-xlsr-ready")

## add audio array

In [19]:

def voice_sep(sig):
    sig = np.array(sig).flatten()
    S_full, phase = librosa.magphase(librosa.stft(sig))
    S_filter = librosa.decompose.nn_filter(S_full,
                                       aggregate=np.median,
                                       metric='cosine',
                                       width=int(librosa.time_to_frames(2, sr=sr)))
    S_filter = np.minimum(S_full, S_filter)
    margin_v = 2
    power = 2
    mask_v = librosa.util.softmask(S_full - S_filter,
                               margin_v * S_filter,
                               power=power)
    S_foreground = mask_v * S_full
    y_foreground = librosa.istft(S_foreground * phase)
    return y_foreground

def load_audio(batch):
    batch['array'],_ = librosa.load('./dataset/audio/'+batch['filename'],sr=16000)
    batch['array'] = np.array(batch['array'])
    
    return batch

In [20]:
audio_data = remove_spectial_char_data.map(load_audio)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [21]:
type(audio_data[0]['array'])

list

In [22]:
type(audio_data[0]['array'][0])

float

## Data augmentation
- RIR applied
- stretching(fast,slow)

In [23]:
rir_raw,_ = librosa.load('./room_component.wav',sr)
rir = torch.from_numpy(rir_raw.reshape(1,-1))
print(rir.shape)

torch.Size([1, 4160])


In [26]:
def rir_applied(batch):

    speech = torch.from_numpy(np.array(batch['array'],dtype=np.float32).reshape(1,-1))

    speech_ = torch.nn.functional.pad(speech, (rir.shape[1] - 1, 0))
    print(speech.dtype)
    print(speech_.dtype)
    print(rir.dtype)
    augmented = torch.nn.functional.conv1d(speech_[None, ...], rir[None, ...])[0]
    batch['array'] = augmented.reshape(-1)
    return batch

def fast_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,0.8)
    return batch

def slow_stretching(batch):
    array = np.array(batch['array'],dtype=np.float32)
    batch['array'] = librosa.effects.time_stretch(array,1.2)
    return batch

In [ ]:
rir_applied_audio_data = audio_data.map(rir_applied)

In [45]:
fast_stretching_data = audio_data.map(fast_stretching)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-f49502427ad3afb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-bd67c8f5c959c293.arrow


In [46]:
slow_stretching_data = audio_data.map(slow_stretching)

Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-f49502427ad3afb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-4226a44a075f52dd.arrow


## Preprocess Data

In [47]:
def prepare_dataset(batch):

    # batched output is "un-batched"
    batch["input_values"] = processor(batch["array"], sampling_rate=16000).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [50]:
ds_list = [audio_data, rir_applied_audio_data, fast_stretching_data, slow_stretching_data]
prepare_ds_list = []
for ds in ds_list:
    prepare_ds_list.append(ds.map(
        prepare_dataset,
        remove_columns=ds.column_names,
        # num_proc=2
    ))

  0%|          | 0/4 [00:00<?, ?it/s]Loading cached processed dataset at C:\Users\AI_server\.cache\huggingface\datasets\csv\default-f49502427ad3afb2\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519\cache-2d4b16bd19083f38.arrow


  0%|          | 0/1000 [00:00<?, ?ex/s]

 50%|█████     | 2/4 [00:18<00:18,  9.15s/it]

  0%|          | 0/1000 [00:00<?, ?ex/s]

 75%|███████▌  | 3/4 [00:41<00:14, 14.99s/it]

  0%|          | 0/1000 [00:00<?, ?ex/s]

100%|██████████| 4/4 [00:56<00:00, 14.23s/it]


In [104]:
augmented_data = torch.utils.data.ConcatDataset(prepare_ds_list)

# Training

## Set-up Trainer

In [8]:
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.cuda.HalfTensor]]]) -> Dict[str, torch.cuda.HalfTensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [9]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [10]:
from datasets import load_dataset, load_metric, Audio

wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

In [11]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

## Import Model

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53',
    # "./wav2vec2-large-xlsr-ko-demo/checkpoint-1900", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

XLSR-Wav2Vec2의 첫 번째 구성 요소는 원시 음성 신호에서 음향적으로 의미가 있지만 문맥적으로 독립적인 기능을 추출하는 데 사용되는 CNN 계층 스택으로 구성됩니다.  
모델의 이 부분은 사전 교육 중에 이미 충분히 훈련되었으며 논문에 명시된 바와 같이 더 이상 미세 조정할 필요가 없습니다. 따라서 특징 추출 부분의 모든 파라미터에 대해 require_grad를 False로 설정할 수 있다.

In [57]:
model.freeze_feature_extractor()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


메모리를 절약하기 위해 그라데이션 체크포인팅을 활성화

In [58]:
model.gradient_checkpointing_enable()

## TrainingArguments

In [59]:
from transformers import TrainingArguments
from transformers import TrainerCallback

tb = transformers.integrations.TensorBoardCallback()
class MyCallback(TrainerCallback):
  def on_evaluate(self, arg, state, control, **kwargs):
    if state.global_step == int(state.best_model_checkpint):
          model.save(os.path.join(arg.output_dir,'best_model'))

training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-large-xlsr-ko-demo",
  output_dir="./wav2vec2-large-xlsr-ko-demo",
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=88,
  fp16=True,
  # save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  callbacks=[tb,]
)

In [107]:
len(augmented_data)

4000

In [108]:
from transformers import Trainer
from torch.utils.data import random_split

ds_size = len(augmented_data)
train_size = int(ds_size*0.8)
val_size = ds_size - train_size
train_ds, val_ds = random_split(augmented_data,[train_size,val_size])

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


## Model Training

In [109]:
torch.cuda.is_available()

True

In [111]:
trainer.train()

c:\Users\AI_server\Anaconda3\envs\STT\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3200
  Num Epochs = 88
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 2
  Total optimization steps = 8800


  0%|          | 0/8800 [00:00<?, ?it/s]

{'loss': 0.4571, 'learning_rate': 5.999999999999999e-06, 'epoch': 0.1}
{'loss': 0.6694, 'learning_rate': 1.1999999999999999e-05, 'epoch': 0.2}
{'loss': 0.4597, 'learning_rate': 1.7999999999999997e-05, 'epoch': 0.3}
{'loss': 0.4644, 'learning_rate': 2.3999999999999997e-05, 'epoch': 0.4}
{'loss': 0.4915, 'learning_rate': 2.9999999999999997e-05, 'epoch': 0.5}
{'loss': 0.4299, 'learning_rate': 3.5999999999999994e-05, 'epoch': 0.6}
{'loss': 0.4021, 'learning_rate': 4.2e-05, 'epoch': 0.7}
{'loss': 0.4114, 'learning_rate': 4.7999999999999994e-05, 'epoch': 0.8}
{'loss': 0.3713, 'learning_rate': 5.399999999999999e-05, 'epoch': 0.9}


***** Running Evaluation *****
  Num examples = 800
  Batch size = 8


{'loss': 0.4654, 'learning_rate': 5.9999999999999995e-05, 'epoch': 1.0}


...

  0%|          | 0/100 [00:00<?, ?it/s]

Saving model checkpoint to ./wav2vec2-large-xlsr-ko-demo\checkpoint-8800
Configuration saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8800\config.json


{'eval_loss': 0.00014746309898328036, 'eval_wer': 0.4257329779500848, 'eval_runtime': 26.6356, 'eval_samples_per_second': 30.035, 'eval_steps_per_second': 3.754, 'epoch': 88.0}


Model weights saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8800\pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-ko-demo\checkpoint-8800\preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-ko-demo\checkpoint-8600] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 13747.81, 'train_samples_per_second': 20.483, 'train_steps_per_second': 0.64, 'train_loss': 0.05770374050787227, 'epoch': 88.0}


CTC 손실을 사용하여 더 큰 데이터 세트에서 더 큰 모델을 미세 조정하려면 [여기서](https://github.com/huggingface/transformers/tree/master/examples/pytorch/speech-recognition#connectionist-temporal-classification-without-language-model-ctc-wo-lm) 공식 음성 인식 예를 살펴봐야 한다.